## Creating timeseries

This notebook goes through the steps of taking the NREL time series data provided to us by Xinshuo and mapping that to the Texas7k dataset given to us by Texas A&M University.

In [65]:
import pandas as pd
import numpy as np
import datetime

# We are going to do some pre-processing to make sure all asset names have underscores instead of spaces
wind_forecast_df = pd.read_csv("./NREL Stuff/wind_day_ahead_forecast_site_2018.csv")
wind_actual_df = pd.read_csv("./NREL Stuff/wind_actual_1h_site_2018.csv")
load_forecast_df = pd.read_csv("./NREL Stuff/load_day_ahead_forecast_zone_2018.csv")
load_actuals_df = pd.read_csv("./NREL Stuff/load_actual_1h_2018.csv")
sol_forecast_df = pd.read_csv("./NREL Stuff/solar_day_ahead_forecast_site_2018.csv")
sol_actual_df = pd.read_csv("./NREL Stuff/solar_actual_1h_site_2018.csv")

wind_mappings = pd.read_csv("./NREL Stuff/Texas7k_NREL_wind_map.csv")


wind_forecast_df.columns = wind_forecast_df.columns.str.replace(' ', '_')
wind_actual_df.columns = wind_actual_df.columns.str.replace(' ', '_')
load_forecast_df.columns = load_forecast_df.columns.str.replace(' ', '_')
load_actuals_df.columns = load_actuals_df.columns.str.replace(' ', '_')
sol_forecast_df.columns = sol_forecast_df.columns.str.replace(' ', '_')
sol_actual_df.columns = sol_actual_df.columns.str.replace(' ', '_')

wind_forecast_df.to_csv("./NREL Stuff/wind_day_ahead_forecast_site_2018.csv")
wind_actual_df.to_csv("./NREL Stuff/wind_actual_1h_site_2018.csv")
load_forecast_df.to_csv("./NREL Stuff/load_day_ahead_forecast_zone_2018.csv")
load_actuals_df.to_csv("./NREL Stuff/load_actual_1h_2018.csv")
sol_forecast_df.to_csv("./NREL Stuff/solar_day_ahead_forecast_site_2018.csv")
sol_actual_df.to_csv("./NREL Stuff/solar_actual_1h_site_2018.csv")


wind_mappings['NREL Wind Site'] = wind_mappings['NREL Wind Site'].str.replace(' ', '_')  # or .replace as above
wind_mappings.to_csv("./NREL Stuff/Texas7k_NREL_wind_map.csv")

In [66]:
# import the files we will need (excluding the timeseries file given to us by Xinshuo)
bus = pd.read_csv("./TX_Data/SourceData/bus.csv")
branch = pd.read_csv("./TX_Data/SourceData/branch.csv")
gen = pd.read_csv("./TX_Data/SourceData/gen.csv")
wind_mappings = pd.read_csv("./NREL Stuff/Texas7k_NREL_wind_map.csv")
solar_mappings = pd.read_csv("./NREL Stuff/Texas7k_NREL_solar_map.csv")

# we will produce forecasts and actuals between the datetimes start_time_local and end_time_local
start_time_local = "2018-01-02 00:00:00"
end_time_local = "2018-12-30 23:00:00"
# these tapering variables are used to trim the actuals to be consistent with the dates above
actuals_taper_front = 30
actuals_taper_back = -18

forecasts_taper_front = 24 # need this for wind and load to equalize the length of the forecasts for each asset type

'''Note: our methodology here relies on the bus numbers being unique for wind and solar assets. Bus numbers
are not unique generally, but they appear to be so for ERCOT w/r/t solar and wind generators. If this is not
the case generally, we will have to be far more careful about how to map them. One way to get around it would be
to store a dictionary with the number of times a non-unique bus has been used, and then use that to index properly
around a subset of length > 1 - old versions of this document (prior to Aug 17 2021) implemented something similar
for wind, so we can revert to that if we need to in the future'''



'Note: our methodology here relies on the bus numbers being unique for wind and solar assets. Bus numbers\nare not unique generally, but they appear to be so for ERCOT w/r/t solar and wind generators. If this is not\nthe case generally, we will have to be far more careful about how to map them. One way to get around it would be\nto store a dictionary with the number of times a non-unique bus has been used, and then use that to index properly\naround a subset of length > 1 - old versions of this document (prior to Aug 17 2021) implemented something similar\nfor wind, so we can revert to that if we need to in the future'

In [67]:
# start with wind forecast for now.... can turn this into a function that applies to all the asset types later
wind_forecast_df = pd.read_csv("./NREL Stuff/wind_day_ahead_forecast_site_2018.csv")
wind_actual_df = pd.read_csv("./NREL Stuff/wind_actual_1h_site_2018.csv")

# change this to just save out the whole year
wind_forecasting_horizon = 24
# we will get some days around the day that we are interested in to populate for this test run
days_before = 1
days_after = 1
day_of_interest = 191 # july 10
#wind_forecast_subset = wind_forecast_df.iloc[np.maximum(0,(day_of_interest-days_before - 1))*wind_forecasting_horizon:np.minimum(365,(day_of_interest+days_after))*wind_forecasting_horizon,:]
#wind_actual_subset = wind_actual_df.iloc[np.maximum(0,(day_of_interest-days_before-1))*wind_forecasting_horizon:np.minimum(365,(day_of_interest+days_after))*wind_forecasting_horizon,:]
# getting the year, month, day, and hour in order to mimic the RTS formatting
#temp = wind_forecast_subset.loc[:,'Forecast_time']
# dates = pd.to_datetime(temp).dt.tz_localize("UTC") - datetime.timedelta(hours=6) #pull 6 hours for date consistency reasons
# wind_forecast_subset.loc[:,'Forecast_time'] = pd.to_datetime(temp.loc[:,:]).

wind_forecast_subset = wind_forecast_df.iloc[forecasts_taper_front:]
wind_forecast_subset = wind_forecast_subset.reset_index()
# has a few extra hours without Forecasts
wind_actual_subset = wind_actual_df.iloc[actuals_taper_front:actuals_taper_back]
# starting the actuals from the indicated date Jan 1 6 am
# recall the times are in UTC, so this actually corresponds to Jan 1, 12 am local
wind_actual_subset = wind_actual_subset.reset_index()
# adjusted for full year -> cut out the hours that lacked forecasts on either end because I don't think it matters -ER
dates = pd.Series(pd.date_range(start=start_time_local, end=end_time_local, freq='H'))
# the above goes from Jan 1 12 am to Dec 30 11 pm to adjust for the fact that the time zones are in UTC but
# Prescient operates in local time
# this has the first 4 columns set up - what remains is to populate with the appropriate time series which correspond
# to the correct assets

In [68]:
# need to generate the forecasts for the appropriate wind assets 
wnd_nm = 'WND (Wind)'
#get the wind_assets of gen
wind_gens = gen[gen['Fuel'] == wnd_nm]
wind_mappings.head()

,Unnamed: 0,Unnamed: 0.1,Texas7k BusNum,Texas7k GenID,Texas7k SubNum,Texas7k Max MW,Texas7k Min MW,EIA-860 Plant Code,EIA-860 Plant Name,EIA-860 Operating Year,EIA-860 Nameplate Capacity (MW),NREL Wind Site,Mapping Status,Distribution Factor,NREL Capacity Proportion,GEN UID
0,0,0,190193,1,3131,253.0,34.10,60902,Dermott Wind,2017,253.0,Amazon_Wind_Farm_Texas,1,1.0,330.0,60902_OnshoreWindTurbine_1
1,1,1,120493,1,1261,99.8,20.75,58000,Anacacho Wind Farm LLC,2012,99.8,Anacacho_Wind_Farm,1,1.0,129.0,58000_OnshoreWindTurbine_1
2,2,2,160281,1,2424,188.0,46.66,57927,Baffin Wind,2014,188.0,Baffin,1,1.0,264.0,57927_OnshoreWindTurbine_1
3,3,3,150496,1,2197,120.0,45.51,57156,Barton Chapel Wind Farm,2009,120.0,Barton_Chapel_Wind_Farm,1,1.0,157.0,57156_OnshoreWindTurbine_1
4,4,4,220216,1,3727,196.7,65.47,59972,Bearkat,2018,196.7,Bearkat_I,1,1.0,257.0,59972_OnshoreWindTurbine_1


In [69]:
# creates a temporary dataframe for the output
temp_output_df_DA = pd.DataFrame({"Year": dates.dt.year, "Month": dates.dt.month, "Day": dates.dt.day, "Period": dates.dt.hour})
temp_output_df_AC = pd.DataFrame({"Year": dates.dt.year, "Month": dates.dt.month, "Day": dates.dt.day, "Period": dates.dt.hour})
# creates a dictionary for the number of times the plant code is used
# this is necessary because we need to make sure we're pulling the correct distribution factor, nrel capacity, and
# texas7k max capacity when scaling in situations where multiple Texas7k generators have the same plant code and
# therefore map from the same NREL wind farm
plant_codes_num_used = {}
gen_codes = np.unique(wind_gens['EIA-860 Plant Code'])
times_used = [0]*len(gen_codes)
plant_codes_num_used = dict(zip(gen_codes, times_used))

# will essentially iterate across the rows of wind_gens
# for wind, the assets are essentially mapped 1:1, or close to it. This will not be the case for solar, so our 
# methodology will change when we get there.
for i in np.arange(wind_gens.shape[0]):
    # finds the gen uid for the associated row, as well as the plant code
    gen_uid = wind_gens.iloc[i]['GEN UID']
    gen_code = wind_gens.iloc[i]['EIA-860 Plant Code']
    gen_bus = wind_gens.iloc[i]['Bus ID']
    # finds the nrel name in wind mappings which agrees with the bus num. this can return lists of length greater than 1
    nrel_name = wind_mappings[wind_mappings['Texas7k BusNum'] == gen_bus]['NREL Wind Site']
    # finds the index of the correct name in wind_mappings so it can accurately pull the distribution and max capacities
    # based on the mappings above, pull the 7k max, NREL capacity, and distribution factor
    # note: we won't have to do all of the below for solar because Majid's mappings already took that into account
    texas7kmax = wind_mappings[wind_mappings['Texas7k BusNum'] == gen_bus]['Texas7k Max MW']
    nrel_capacity = wind_mappings[wind_mappings['Texas7k BusNum'] == gen_bus]['NREL Capacity Proportion']
    dist_factor = wind_mappings[wind_mappings['Texas7k BusNum'] == gen_bus]['Distribution Factor']
    # will multiply the forecast by the below to scale it for texas 7k
    forecast_multiplier = float(dist_factor / nrel_capacity * texas7kmax)
    # assign to the output dataframe
    tst = wind_forecast_subset[nrel_name] * forecast_multiplier
    temp_output_df_DA[gen_uid] = wind_forecast_subset[nrel_name] * forecast_multiplier
    temp_output_df_AC[gen_uid] = wind_actual_subset[nrel_name] * forecast_multiplier
    plant_codes_num_used[gen_code] += 1
temp_output_df_DA.to_csv("./TX_Data/timeseries_data_files/WIND/DAY_AHEAD_wind.csv", index=False)
temp_output_df_AC.to_csv("./TX_Data/timeseries_data_files/WIND/REAL_TIME_wind.csv", index=False)

Now we will handle the load forecasts. This is slightly trickier, as NREL provides 48 hours of forecasts for loads, as opposed to 24. This means that days are double-forecasted, and we have to be careful to make sure we are always pulling from the correct forecast time.

In [70]:
load_forecast_df = pd.read_csv("./NREL Stuff/load_day_ahead_forecast_zone_2018.csv")
load_forecasting_horizon = 24
hours_in_day = 24
days_after_load = 1
# delete the first forecast for load
load_forecast_df = load_forecast_df.drop_duplicates(subset=['Forecast_time'], keep="last")
# dup = load_forecast_df.duplicated(subset = "Forecast_time", keep="last")
# load_forecast_df = load_forecast_df[~dup.values]
# load_forecast_subset = load_forecast_df.iloc[np.maximum(0,(day_of_interest-days_before - 1))*load_forecasting_horizon:np.minimum(365,(day_of_interest+days_after_load))*load_forecasting_horizon,:]
# temp = load_forecast_subset.loc[:,'Forecast_time']
# dates = pd.to_datetime(temp).dt.tz_localize("UTC")- datetime.timedelta(hours=6) #pull 6 hours for date consistency reasons
# wind_forecast_subset.loc[:,'Forecast_time'] = pd.to_datetime(temp.loc[:,:]).
load_forecast_subset = load_forecast_df.iloc[forecasts_taper_front:]
load_forecast_subset = load_forecast_subset.drop("Issue_time", axis=1)
load_forecast_subset = load_forecast_subset.reset_index()
dates = pd.Series(pd.date_range(start=start_time_local, end=end_time_local, freq='H'))
year = dates.dt.year
month = dates.dt.month
day = dates.dt.day
hours = dates.dt.hour#*int(load_forecasting_horizon/hours_in_day))
load_output_df_DA = pd.DataFrame({'Year': year, 'Month':month, 'Day':day, 'Period':hours})
# this has the first 4 columns set up - what remains is to populate with the appropriate time series which correspond
# to the correct assets
zones = load_forecast_subset.columns[2:]
load_output_df_DA[zones] = load_forecast_subset.loc[:, zones]
print(load_output_df_DA.head(50))
load_output_df_DA.to_csv("./TX_Data/timeseries_data_files/LOAD/DAY_AHEAD_regional_Load.csv", index=False)

    Year  Month  Day  Period     Forecast_time        Coast         East  \
0   2018      1    2       0   2018-01-02 6:00  11539.93652  2225.211670   
1   2018      1    2       1   2018-01-02 7:00  11872.31836  2182.493652   
2   2018      1    2       2   2018-01-02 8:00  11795.97461  2143.105957   
3   2018      1    2       3   2018-01-02 9:00  12022.63086  2146.737305   
4   2018      1    2       4  2018-01-02 10:00  12673.70313  2357.272705   
5   2018      1    2       5  2018-01-02 11:00  13588.69824  2423.565674   
6   2018      1    2       6  2018-01-02 12:00  13552.07031  2529.919922   
7   2018      1    2       7  2018-01-02 13:00  13232.78320  2470.176514   
8   2018      1    2       8  2018-01-02 14:00  13427.34668  2381.910400   
9   2018      1    2       9  2018-01-02 15:00  13563.33008  2293.909912   
10  2018      1    2      10  2018-01-02 16:00  12887.97461  2234.633789   
11  2018      1    2      11  2018-01-02 17:00  13384.44043  2182.863281   
12  2018    

Now we will do load actuals.

In [71]:
load_actuals_df = pd.read_csv("./NREL Stuff/load_actual_1h_2018.csv")
# load_actuals_horizon = 24
# days_after_load_actuals = 0
# shift = 6 # shift because of a slight inconsistency; these files go from 6 pm Dec 31 2017 to 6 pm Dec 31 2018 (in local time)
# load_actuals_subset = load_actuals_df.iloc[shift + np.maximum(0,(day_of_interest-days_before - 1))*load_actuals_horizon: shift + np.minimum(365,(day_of_interest+days_after_load_actuals))*load_actuals_horizon,:]
# temp = load_actuals_subset.loc[:,'Time']
# dates = pd.to_datetime(temp).dt.tz_localize("UTC") - datetime.timedelta(hours=6)
# once again, the actuals span a longer time than the forecasts, so we taper 
load_actuals_subset = load_actuals_df.iloc[actuals_taper_front:actuals_taper_back].reset_index()
year = dates.dt.year
month = dates.dt.month
day = dates.dt.day
hours = dates.dt.hour
load_output_df_RT = pd.DataFrame({'Year': year, 'Month':month, 'Day':day, 'Period':hours})
# this has the first 4 columns set up - what remains is to populate with the appropriate time series which correspond
# to the correct assets
zones = load_actuals_subset.columns[2:]
load_output_df_RT[zones] = load_actuals_subset.loc[:, zones]
load_output_df_RT.to_csv("./TX_Data/timeseries_data_files/LOAD/REAL_TIME_regional_Load.csv", index=False)

### Solar

In [72]:

# start with solar forecast for now.... can turn this into a function that applies to all the asset types later
sol_forecast_df = pd.read_csv("./NREL Stuff/solar_day_ahead_forecast_site_2018.csv")
sol_actual_df = pd.read_csv("./NREL Stuff/solar_actual_1h_site_2018.csv")
sol_forecasting_horizon = 24

# ignored for the purposes
# we will get some days around the day that we are interested in to populate for this test run
days_before = 1
days_after = 1
day_of_interest = 190 # july 10
#sol_forecast_subset = sol_forecast_df.iloc[np.maximum(0,(day_of_interest-days_before - 1))*sol_forecasting_horizon:np.minimum(365,(day_of_interest+days_after))*sol_forecasting_horizon,:]
#sol_actual_subset = sol_actual_df.iloc[np.maximum(0,(day_of_interest-days_before-1))*sol_forecasting_horizon:np.minimum(365,(day_of_interest+days_after))*sol_forecasting_horizon,:]

sol_forecast_subset = sol_forecast_df # don't need to use the forecast taper here like we did for wind/load

sol_actual_subset = sol_actual_df.iloc[actuals_taper_front:actuals_taper_back]
sol_actual_subset = sol_actual_subset.reset_index()


# getting the year, month, day, and hour in order to mimic the RTS formatting
#temp = sol_forecast_subset.loc[:,'Forecast_time']
#dates = pd.to_datetime(temp).dt.tz_localize("UTC") - datetime.timedelta(hours=6) #pull 6 hours for date consistency reasons

dates = pd.Series(pd.date_range(start=start_time_local, end=end_time_local, freq='H'))

year = dates.dt.year
month = dates.dt.month
day = dates.dt.day
hours = dates.dt.hour
sol_output_df_DA = pd.DataFrame({'Year': year, 'Month':month, 'Day':day, 'Period':hours})
# this has the first 4 columns set up - what remains is to populate with the appropriate time series which correspond
# to the correct assets

In [73]:
# need to generate the forecasts for the appropriate solar assets 
sol_nm = 'SUN (Solar)'
#get the solar assets of gen
sol_gens = gen[gen['Fuel'] == sol_nm]
# creates a temporary dataframe for the output
temp_output_df_DA = pd.DataFrame({'Year': year, 'Month':month, 'Day':day, 'Period':hours})
temp_output_df_AC = pd.DataFrame({'Year': year, 'Month':month, 'Day':day, 'Period':hours})

# will essentially iterate across the rows of sol_gens
# note that for ERCOT, we have only one generator at each bus, so there is no possibility of a 
for i in np.arange(sol_gens.shape[0]):
    # finds the gen uid for the associated row, as well as the plant code and bus ID
    gen_uid = sol_gens.iloc[i]['GEN UID']
    bus_id = sol_gens.iloc[i]['Bus ID']
    # finds the nrel name in solar mappings which agrees with the bus id.
    nrel_name = solar_mappings[solar_mappings['BusNum'] == bus_id]['Min_site']
    # gets the appropriate dist factor - note that in this case, since we are using Majid's solar mappings, the dist
    # factor alone is sufficient for us to map the solars
    nrel_dist_factor = solar_mappings[solar_mappings['BusNum'] == bus_id]['dist_factor']
    print(nrel_dist_factor)
    # in the case that it is assets that have no NREL map then output zeroes
    if (nrel_name.empty):
        temp_output_df_DA[gen_uid] = np.zeros(len(sol_forecast_subset))
        temp_output_df_AC[gen_uid] = np.zeros(len(sol_actual_subset))
        continue
    # assign to the output dataframe
    temp_output_df_DA[gen_uid] = sol_forecast_subset[nrel_name] * float(nrel_dist_factor)
    temp_output_df_AC[gen_uid] = sol_actual_subset[nrel_name] * float(nrel_dist_factor)
    plant_codes_num_used[gen_code] += 1
temp_output_df_DA.to_csv("./TX_Data/timeseries_data_files/PV/DAY_AHEAD_pv.csv", index=False)
temp_output_df_AC.to_csv("./TX_Data/timeseries_data_files/PV/REAL_TIME_pv.csv", index=False)

30    1.087
Name: dist_factor, dtype: float64
21    20.2
Name: dist_factor, dtype: float64
28    0.7913
Name: dist_factor, dtype: float64
22    0.7826
Name: dist_factor, dtype: float64
20    15.75
Name: dist_factor, dtype: float64
25    0.6522
Name: dist_factor, dtype: float64
24    0.5152
Name: dist_factor, dtype: float64
19    0.5043
Name: dist_factor, dtype: float64
8    3.367
Name: dist_factor, dtype: float64
33    0.4348
Name: dist_factor, dtype: float64
32    0.4348
Name: dist_factor, dtype: float64
31    10.0
Name: dist_factor, dtype: float64
23    0.4348
Name: dist_factor, dtype: float64
6    1.3336
Name: dist_factor, dtype: float64
7    1.3333
Name: dist_factor, dtype: float64
9    0.983
Name: dist_factor, dtype: float64
3    0.4423
Name: dist_factor, dtype: float64
14    2.2727
Name: dist_factor, dtype: float64
13    0.117
Name: dist_factor, dtype: float64
12    0.117
Name: dist_factor, dtype: float64
11    0.117
Name: dist_factor, dtype: float64
35    0.117
Name: dist_factor

What remains:
* Verify that these are working properly (correct format of output, make sure output reconciles with what it should be, and corresponds to the correct rows)
* Verify understanding of time zones - ideally, we should not have any conversion, but in RTS as currently set up, it seems somewhat necessary. We can potentially change this ourselves dow the line
